# MySQL 8.0.40

## 0) 前提

* エンジン: **MySQL 8**
* 並び順: 任意（`ORDER BY` を付けない）
* `NOT IN` は NULL 罠のため回避
* 判定は **ID 基準**、表示は仕様どおりの列名と順序

## 1) 問題

* `RequestAccepted` から **友だち数が最大の人**とその**友だち数**を求める
  （リクエストは相互に友だちとみなす＝無向グラフ）
* 入力テーブル例: `RequestAccepted(requester_id, accepter_id, accept_date)`
* 出力仕様: `id, num`

  * `id`: 友だち数が最大の人のユーザーID
  * `num`: その人の友だち数
  * 追記: 実運用では同率最大が複数あり得るため、それら全員を返せるとよい

## 2) 最適解（単一クエリ）

> 片方向の受理を **相互のエッジ**に展開してから **人数を集計**し、**最大のみ**を `DENSE_RANK` で抽出。

```sql
WITH edges AS (
  -- 無向化：両方向に展開
  SELECT requester_id AS id, accepter_id AS friend_id FROM RequestAccepted
  UNION ALL
  SELECT accepter_id AS id, requester_id AS friend_id FROM RequestAccepted
),
deg AS (
  -- 各ユーザーの友だち数を集計
  SELECT
    id,
    COUNT(*) AS num
    -- PK(requester_id, accepter_id) により edges 側で重複は発生しないため DISTINCT 不要
  FROM edges
  GROUP BY id
),
ranked AS (
  SELECT
    id,
    num,
    DENSE_RANK() OVER (ORDER BY num DESC) AS rnk
  FROM deg
)
SELECT
  id,
  num
FROM ranked
WHERE rnk = 1;

Runtime 270 ms
Beats 89.32%

```

* 仕様上「最大が一人」と保証されるケースでは 1 行だけ返る
* **Follow up（同率最大を全員）**にもそのまま対応（`rnk = 1`）

## 3) 代替解

> ウィンドウが使えない／避けたい場合は、`MAX` をサブクエリで求めて `HAVING`。

```sql
WITH edges AS (
  SELECT requester_id AS id, accepter_id AS friend_id FROM RequestAccepted
  UNION ALL
  SELECT accepter_id AS id, requester_id AS friend_id FROM RequestAccepted
),
deg AS (
  SELECT id, COUNT(*) AS num
  FROM edges
  GROUP BY id
)
SELECT d.id, d.num
FROM deg AS d
WHERE d.num = (
  SELECT MAX(num) FROM deg
);

Runtime 296 ms
Beats 54.40%

```

## 4) 要点解説

* **無向化がコア**：1 レコードを両方向へ `UNION ALL` 展開し、各 `id` の次数（友だち数）を数える
* **重複対策**：テーブルの PK が `(requester_id, accepter_id)` のため、両方向展開しても同一方向の重複は発生しない。一般化するなら `COUNT(DISTINCT friend_id)` でも可
* **同率最大への拡張**：`DENSE_RANK() OVER (ORDER BY num DESC) = 1` で全員抽出

## 5) 計算量（概算）

* `edges` 作成：入力件数を `N` とすると **O(N)**（2N 行）
* `GROUP BY` 集計：**O(N)**～**O(N log N)**（実装とインデックス依存）
* `DENSE_RANK`：パーティション全体で **O(U log U)**（`U` はユーザー数）

## 6) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[RequestAccepted] --> B[両方向に展開 edges]
  B --> C[ユーザー単位で集計 deg]
  C --> D[ウィンドウで順位付け ranked]
  D --> E[順位1のみ抽出]
  E --> F[出力 id と num]
```
# MySQL 8.0.40

## 0) 前提

* エンジン: **MySQL 8**
* 並び順: 任意（最終結果の並びは不問）
* `NOT IN` は使わない
* 判定は **ID 基準**

## 1) 問題

* `RequestAccepted` から **友だち数が最大の人**とその**友だち数**を求める
* 入力: `RequestAccepted(requester_id, accepter_id, accept_date)`（PK: `(requester_id, accepter_id)`）
* 出力: `id, num`（同率最大が複数でも全員返せると尚良い）

## 2) 最適解（単一クエリ）

あなたの解法はロジック的に正しく、一般的な答えとして十分高速です。
さらに**中間データ量を減らす**形にすると大規模データで伸びやすいです。

### 改善案 A：事前集計→合算（2 段集約で edges の 2N 行を作らない）

```sql
WITH deg_parts AS (
  SELECT requester_id AS id, COUNT(*) AS cnt
  FROM RequestAccepted
  GROUP BY requester_id
  UNION ALL
  SELECT accepter_id AS id, COUNT(*) AS cnt
  FROM RequestAccepted
  GROUP BY accepter_id
),
deg AS (
  SELECT id, SUM(cnt) AS num
  FROM deg_parts
  GROUP BY id
)
SELECT id, num
FROM deg
WHERE num = (SELECT MAX(num) FROM deg);

Runtime 290 ms
Beats 62.27%

```

* ポイント: `edges` を作らず、**各方向で一度だけ集計**→最後に合算
* 多くのケースでメモリ・ソート量が減り、**CTE 1 本分少ない**分だけ実行計画が素直になります
* もし「1 人だけでよい」なら `ORDER BY num DESC LIMIT 1` も可（順序不要条件に抵触しない範囲で）

### 改善案 B：**対称重複（両方向の受理が存在）**まで堅牢にする

実データで `(1,2)` と `(2,1)` の**両方が存在**し得るなら、現在の `UNION ALL` は**二重カウント**になります。
その場合は**無向ペアを正規化（LEAST/GREATEST）して一意化**してから次数を数えます。

```sql
WITH uniq_pairs AS (
  SELECT
    LEAST(requester_id, accepter_id) AS u,
    GREATEST(requester_id, accepter_id) AS v
  FROM RequestAccepted
  GROUP BY LEAST(requester_id, accepter_id), GREATEST(requester_id, accepter_id)
),
deg AS (
  SELECT u AS id, COUNT(*) AS num FROM uniq_pairs GROUP BY u
  UNION ALL
  SELECT v AS id, COUNT(*) AS num FROM uniq_pairs GROUP BY v
),
sumdeg AS (
  SELECT id, SUM(num) AS num FROM deg GROUP BY id
)
SELECT id, num
FROM sumdeg
WHERE num = (SELECT MAX(num) FROM sumdeg);

Runtime 269 ms
Beats 90.43%

```

* これで**方向違いの重複**があっても「友だち 1 人」として正しく 1 回だけ数えます
* LeetCode 前提では不要なことも多いですが、**実運用**では安全

## 3) 代替解

> 「最大が 1 人だけで良い」前提をフル活用して最短経路

```sql
WITH deg_parts AS (
  SELECT requester_id AS id, COUNT(*) AS cnt
  FROM RequestAccepted GROUP BY requester_id
  UNION ALL
  SELECT accepter_id AS id, COUNT(*) AS cnt
  FROM RequestAccepted GROUP BY accepter_id
),
deg AS (
  SELECT id, SUM(cnt) AS num FROM deg_parts GROUP BY id
)
SELECT id, num
FROM deg
ORDER BY num DESC
LIMIT 1;

Runtime 279 ms
Beats 77.14%

```

* `ORDER BY ... LIMIT 1` はソート最適化（filesort の早期打ち切り）が効きやすく、**最大 1 行**の取得に強いです

## 4) 要点解説

* **中間行を減らす**: `edges` で 2N 行に増やすより、**先に方向別で集計**→合算の方が軽くなることが多い
* **対称重複への備え**: 実務では `(a,b)` と `(b,a)` の両在は珍しくないため、`LEAST/GREATEST` で正規化してから次数カウントすると堅牢
* **同率最大対応**: `WHERE num = (SELECT MAX(num) ...)` ならそのまま**全員**返せる
* **ウィンドウ関数 vs サブクエリ**: MySQL 8 では**どちらも最適化が効く**。今回のような「最大のみ抽出」なら `MAX` サブクエリが計画上シンプル

## 5) 物理設計（インデックス最適化）

* 既存 PK: `(requester_id, accepter_id)`

  * `GROUP BY requester_id` は**左端一致**で効く
  * しかし `GROUP BY accepter_id` は効きにくい → **セカンダリインデックス**を推奨

    ```sql
    CREATE INDEX ix_requestaccepted_accepter ON RequestAccepted (accepter_id);
    ```
  * 改善案 B の正規化を**高頻度で使う場合**は、余裕があれば**生成列 + インデックス**も選択肢

    ```sql
    ALTER TABLE RequestAccepted
      ADD COLUMN u INT AS (LEAST(requester_id, accepter_id)) VIRTUAL,
      ADD COLUMN v INT AS (GREATEST(requester_id, accepter_id)) VIRTUAL,
      ADD INDEX ix_reqacc_u_v (u, v);
    ```

    ※ スキーマ変更が許される実環境向け。LeetCode 等では不可

## 6) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[RequestAccepted] --> B[方向別に集計 requester_id 側]
  A --> C[方向別に集計 accepter_id 側]
  B --> D[合算 SUM で次数]
  C --> D
  D --> E[MAX を比較して抽出]
  E --> F[出力 id と num]
```

---

### まとめ（実務チューニングの優先度）

1. **`ix_requestaccepted_accepter(accepter_id)` を追加**
2. クエリは **改善案 A（2 段集約）** を基準に、必要なら **B（正規化）**
3. 「最大が 1 人だけで良い」なら **`ORDER BY num DESC LIMIT 1`** 版で最短取得

あなたの 270ms は既に速い部類ですが、上記の **2 段集約 + セカンダリインデックス** でさらなる短縮が見込めます。
